In [4]:
import pandas as pd

In [5]:
# !scp mygpu:TXL/transformer-xl/pytorch/test_res.csv ./test_res_retrieval.csv

In [6]:
# !scp gpu6:bulatov/TXL/_git/test_res.csv ./test_res_synthetic_results.csv

In [7]:
# df = pd.read_csv('test_res_synthetic.csv')
# df = pd.concat((df, pd.read_csv('test_res_mygpu.csv')))
# df = pd.concat((df, pd.read_csv('test_res_synthetic_results.csv')))
# df = pd.concat((df, pd.read_csv('test_res_synthetic_results_all.csv')))
df_retrieval = pd.read_csv('test_res_retrieval.csv')
df = pd.read_csv('test_res_synthetic_results.csv')

In [8]:
df = df.dropna(subset=['last_val_acc'])
df = df.drop('work_dir', axis=1)
df = df.drop_duplicates()

df = df[(df.max_step > 450000) | (df.tgt_len >= 24)]
# df = df[~((df.tgt_len == 12) & (df.max_step == 700000))]
df = df[~((df.tgt_len == 24) & (df.max_step == 300000))]
df = df[~((df.tgt_len == 8) & (df.num_mem_tokens == 12))]
df = df[~((df.dataset == 'reverse') & (df.max_step == 900000))]


df_retrieval = df_retrieval.dropna(subset=['last_val_acc'])
df_retrieval = df_retrieval.drop('work_dir', axis=1)
df_retrieval = df_retrieval.drop_duplicates()

In [9]:
reverse = df[df.dataset == 'reverse']
copy = df[df.dataset == 'copy']
retrieval = df_retrieval[df_retrieval.dataset.apply(lambda x: 'retrieval' in x)]

In [10]:
gb_cols = ['dataset', 'tgt_len', 'lr', 'max_step',  'num_mem_tokens', 'mem_len', 'mem_backprop_depth']#, 'seed']
df_ = copy
gb = df_.dropna(subset=['test acc']).groupby(gb_cols).agg({'test acc': tuple})
gb['mean acc'] = df_.groupby(gb_cols).mean()['test acc']
gb

test acc  \
dataset tgt_len lr     max_step num_mem_tokens mem_len mem_backprop_depth                                        
copy    8       0.0001 900000   0              0       0                   (0.102, 0.102, 0.102, 0.101, 0.102)   
                                               8       0                          (0.451, 0.189, 0.259, 0.186)   
                                8              0       0                             (1.0, 1.0, 1.0, 1.0, 1.0)   
        12      0.0001 700000   0              6       0                          (0.657, 0.298, 0.355, 0.856)   
                                6              0       0                                  (1.0, 1.0, 1.0, 1.0)   
                       900000   0              0       0                          (0.102, 0.102, 0.102, 0.102)   
                                               12      0                            (1.0, 0.904, 0.998, 0.998)   
                                12             0       0                                       (1.0, 1.0, 1.0)   
        24      0.0001 400000   0              0       0                                   (0.12, 0.12, 0.115)   
                                               24      0                                     (1.0, 1.0, 0.999)   
                                24             0       0                                       (1.0, 1.0, 1.0)   
                       500000   12             0       0                                            (1.0, 1.0)   
        72      0.0001 400000   0              0       0                                       (1.0, 1.0, 1.0)   

                                                                           mean acc  
dataset tgt_len lr     max_step num_mem_tokens mem_len mem_backprop_depth            
copy    8       0.0001 900000   0              0       0                   0.101800  
                                               8       0                   0.271250  
                                8              0       0                   1.000000  
        12      0.0001 700000   0              6       0                   0.541500  
                                6              0       0                   1.000000  
                       900000   0              0       0                   0.102000  
                                               12      0                   0.975000  
                                12             0       0                   1.000000  
        24      0.0001 400000   0              0       0                   0.118333  
                                               24      0                   0.999667  
                                24             0       0                   1.000000  
                       500000   12             0       0                   1.000000  
        72      0.0001 400000   0              0       0                   1.000000

In [11]:
df_ = reverse
gb_cols = ['dataset', 'tgt_len', 'max_step',  'num_mem_tokens', 'mem_len', 'mem_backprop_depth']#, 'seed']
gb = df_.dropna(subset=['test acc']).groupby(gb_cols).agg({'test acc': tuple})
gb['mean acc'] = df_.groupby(gb_cols).mean()['test acc']
gb['med acc'] = df_.groupby(gb_cols).median()['test acc']
gb['med acc'] = df_.groupby(gb_cols).median()['test acc']
gb

test acc  \
dataset tgt_len max_step num_mem_tokens mem_len mem_backprop_depth                                       
reverse 8       700000   0              0       0                         (0.101, 0.102, 0.102, 0.102)   
                                        8       0                           (0.627, 0.696, 0.68, 0.66)   
                         8              0       0                   (1.0, 0.701, 0.702, 1.0, 1.0, 1.0)   
        12      850000   0              0       0                               (0.1, 0.1, 0.102, 0.1)   
                                        6       0                           (0.832, 0.8, 0.782, 0.784)   
                                        12      0                                (0.877, 0.872, 0.833)   
                         6              0       0                                      (1.0, 1.0, 1.0)   
                         12             0       0                                      (1.0, 1.0, 1.0)   
        24      400000   0              0       0                                (0.102, 0.102, 0.102)   
                                        24      0                                (0.959, 0.899, 0.394)   
                         24             0       0                                 (1.0, 1.0, 1.0, 1.0)   
                500000   0              12      0                                       (0.974, 0.996)   
                         12             0       0                                           (1.0, 1.0)   
        48      250000   0              0       0                                      (1.0, 1.0, 1.0)   

                                                                    mean acc  \
dataset tgt_len max_step num_mem_tokens mem_len mem_backprop_depth             
reverse 8       700000   0              0       0                   0.101750   
                                        8       0                   0.665750   
                         8              0       0                   0.900500   
        12      850000   0              0       0                   0.100500   
                                        6       0                   0.799500   
                                        12      0                   0.860667   
                         6              0       0                   1.000000   
                         12             0       0                   1.000000   
        24      400000   0              0       0                   0.102000   
                                        24      0                   0.750667   
                         24             0       0                   1.000000   
                500000   0              12      0                   0.985000   
                         12             0       0                   1.000000   
        48      250000   0              0       0                   1.000000   

                                                                    med acc  
dataset tgt_len max_step num_mem_tokens mem_len mem_backprop_depth           
reverse 8       700000   0              0       0                     0.102  
                                        8       0                     0.670  
                         8              0       0                     1.000  
        12      850000   0              0       0                     0.100  
                                        6       0                     0.792  
                                        12      0                     0.872  
                         6              0       0                     1.000  
                         12             0       0                     1.000  
        24      400000   0              0       0                     0.102  
                                        24      0                     0.899  
                         24             0       0                     1.000  
                500000   0              12      0                     0.985  
                        

In [12]:
gb_cols = ['dataset', 'tgt_len', 'max_step',  'num_mem_tokens', 'mem_len', 'mem_backprop_depth']
gb = df.dropna(subset=['last_val_acc']).sort_values('last_val_acc', ascending=False).groupby(gb_cols).agg({'last_val_acc': tuple, 'test acc': tuple})

gb['mean acc'] = df.groupby(gb_cols).mean()['last_val_acc']
gb

last_val_acc  \
dataset tgt_len max_step num_mem_tokens mem_len mem_backprop_depth                                                      
copy    8       700000   0              8       0                                                              (0.1,)   
                900000   0              0       0                                   (0.101, 0.1, 0.099, 0.098, 0.098)   
                                        8       0                                   (0.447, 0.258, 0.188, 0.185, 0.1)   
                         8              0       0                                      (1.0, 1.0, 1.0, 1.0, 1.0, 1.0)   
        12      700000   0              6       0                                 (0.808, 0.658, 0.334, 0.285, 0.103)   
                         6              0       0                                           (1.0, 1.0, 1.0, 1.0, 0.1)   
                900000   0              0       0                                          (0.102, 0.101, 0.1, 0.099)   
                                        12      0                                          (1.0, 0.998, 0.981, 0.866)   
                         12             0       0                                              (1.0, 1.0, 1.0, 0.101)   
        24      400000   0              0       0                                               (0.118, 0.116, 0.114)   
                                        24      0                                                   (1.0, 1.0, 0.998)   
                         24             0       0                                                     (1.0, 1.0, 1.0)   
                500000   12             0       0                                                          (1.0, 1.0)   
        72      400000   0              0       0                                                     (1.0, 1.0, 1.0)   
reverse 8       700000   0              0       0                                          (0.103, 0.101, 0.1, 0.099)   
                                        8       0                                        (0.694, 0.678, 0.658, 0.622)   
                         8              0       0                   (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.701, 0.699, 0...   
        12      850000   0              0       0                                        (0.104, 0.103, 0.097, 0.094)   
                                        6       0                           (0.868, 0.844, 0.804, 0.787, 0.779, 0.66)   
                                        12      0                                               (0.873, 0.837, 0.786)   
                         6              0       0                                                     (1.0, 1.0, 1.0)   
                         12             0       0                                                     (1.0, 1.0, 1.0)   
        24      250000   0              24      0                                                            (0.991,)   
                400000   0              0       0                                                 (0.102, 0.101, 0.1)   
                                        24      0                                         (0.943, 0.883, 0.81, 0.553)   
                         24             0       0                                                (1.0, 1.0, 1.0, 1.0)   
                500000   0              12      0                                                       (0.993, 0.98)   
                         12             0       0                                                          (1.0, 1.0)   
        48      250000   0              0       0                                                     (1.0, 1.0, 1.0)   

                                                                                                             test acc  \
dataset tgt_len max_step num_mem_tokens mem_len mem_backprop_depth                                                      
copy    8       700000   0              8       0                                                              

In [13]:
df_ = retrieval[retrieval.lr == 0.0001]
gb_cols = ['dataset', 'tgt_len', 'max_step',  'num_mem_tokens', 'mem_len', 'mem_backprop_depth']#, 'seed']
gb = df_.dropna(subset=['test acc']).groupby(gb_cols).agg({'test acc': tuple})
gb['mean acc'] = df_.groupby(gb_cols).mean()['test acc']
gb

test acc  \
dataset         tgt_len max_step num_mem_tokens mem_len mem_backprop_depth                                                     
retrieval       1       1000000  3              0       1                                                           (0.099,)   
                2       1000000  0              0       0                                                           (0.099,)   
                                                2       0                                              (0.965, 0.999, 0.999)   
                                 2              0       0                                              (0.989, 0.922, 0.984)   
                                                        1                                                           (0.866,)   
                3       1000000  0              0       0                                                           (0.099,)   
                                                3       0                                              (0.669, 0.905, 0.973)   
                                 3              0       0                                              (0.986, 0.994, 0.996)   
                                                        1                                                 (0.995, 0.96, 0.6)   
                4       1000000  0              0       0                                                           (0.496,)   
                                                4       0                                                    (1.0, 1.0, 1.0)   
                                 4              0       0                                                (0.999, 1.0, 0.999)   
                                                        1                                                (1.0, 0.999, 0.999)   
                5       700000   5              0       0                                                           (0.963,)   
                        1000000  0              0       0                                                           (0.496,)   
                                                5       0                                                    (1.0, 1.0, 1.0)   
                                 5              0       0                                              (0.999, 0.998, 0.998)   
                                                        1                                                  (0.999, 1.0, 1.0)   
                10      250000   0              0       0                   (0.228, 0.247, 0.254, 0.237, 0.256, 0.254, 0.22)   
                        400000   0              0       0                                                         (1.0, 1.0)   
retrieval29_ext 60      850000   0              0       0                                              (0.189, 0.192, 0.189)   
retrieval59     120     500000   0              0       0                                                           (0.159,)   
                        600000   0              0       0                                                           (0.171,)   
                        1000000  0              0       0                                                           (0.166,)   
retrieval59_ext 120     600000   0              0       0                                                           (0.152,)   

                                                                            mean acc  
dataset         tgt_len max_step num_mem_tokens mem_len mem_backprop_depth            
retrieval       1       1000000  3              0       1                   0.099000  
                2       1000000  0              0       0                   0.099000  
                                                2       0                   0.987667  
                                 2              0       0                   0.965000  
                                                        1                   0.866000  
                3       1000000  0              0 

In [157]:
df_ = copy[(copy.tgt_len == 12) & (copy.num_mem_tokens == 12)]
df_[[col for col in df_.columns if df_[col].unique().shape[0] > 1]]

,seed,test loss,test ppl,test acc,last_val_acc
3,1234,0.00000,1.0000,1.000,1.000
4,2022,NaN,NaN,NaN,0.302
34,2022,1.27346,3.5732,0.302,0.300
76,2049,NaN,NaN,NaN,0.098
118,2049,NaN,NaN,NaN,1.000


In [158]:
# df_ = reverse[(reverse.num_mem_tokens == 24) & (reverse.tgt_len == 24)]
df_ = reverse[(reverse.num_mem_tokens == 12) & (reverse.tgt_len == 12)]
df_[[col for col in df_.columns if df_[col].unique().shape[0] > 1]]#.sort_values('last_val_acc')#.dropna(subset=['test acc'])

,seed,test loss,test ppl,test acc,last_val_acc
12,2049,NaN,NaN,NaN,0.103
13,1212,0.00000,1.00000,1.000,1.000
14,2023,1.35207,3.86542,0.273,0.268
48,2049,NaN,NaN,NaN,1.000
132,2049,0.00001,1.00001,1.000,1.000


In [98]:
df_ = reverse[(reverse.mem_len == ) & (reverse.tgt_len == 8)]
df_[[col for col in df_.columns if df_[col].unique().shape[0] > 1]]#.sort_values('last_val_acc')#.dropna(subset=['test acc'])

""


In [102]:
df_ = copy[(copy.num_mem_tokens == 12) & (copy.tgt_len == 12)]
df_[[col for col in df_.columns if df_[col].unique().shape[0] > 1]]#.sort_values('last_val_acc')#.dropna(subset=['test acc'])

,lr,lr_min,test loss,test ppl,test acc,last_val_acc
115,0.00010,0.000000,2.30265,10.00066,0.102,0.1
116,0.00025,0.000001,NaN,NaN,NaN,1.0


In [83]:
df_.loc[107]

data              /home/bulatov/bulatov/TXL/data24/data24
dataset                                           reverse
n_layer                                                 4
n_head                                                  4
d_head                                                 64
                                   ...                   
n_nonemb_param                                     921088
test loss                                             0.0
test ppl                                              1.0
test acc                                              1.0
last_val_acc                                          1.0
Name: 107, Length: 73, dtype: object